## Twitter Interests Model from Reddit Data

Import libraries

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from keras.layers import Input, Lambda, Dense, LSTM
from keras.models import Model

Training and testing data

In [7]:
# read data
df = pd.read_csv("data/ml_reddit_data_final.csv").drop(columns = ["Unnamed: 0"])

# separate between training and testing data
X = df[["text"]]
y = df["category"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.01, random_state=42)

train_text = X_train["text"].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

test_text = X_test["text"].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(y_train), dtype = np.int8)
test_label = np.asarray(pd.get_dummies(y_test), dtype = np.int8)

Universal Sentence Encoder

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Tensorflow model

In [ ]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,
	output_shape=(512,))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(23, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', 
	optimizer='adam', metrics=['accuracy'])

Fit model

In [25]:
history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=32,
)

Epoch 1/10


/Users/toni/opt/anaconda3/envs/my-env/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Reshape:0", shape=(None, 320), dtype=float32), dense_shape=Tensor("gradients/EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1604/1604 [==============================] - 33s 16ms/step - loss: 0.8813 - accuracy: 0.7607 - val_loss: 0.6936 - val_accuracy: 0.7881
Epoch 2/10
1604/1604 [==============================] - 26s 16ms/step - loss: 0.6070 - accuracy: 0.8169 - val_loss: 0.6859 - val_accuracy: 0.8035
Epoch 3/10
1604/1604 [==============================] - 25s 15ms/step - loss: 0.5741 - accuracy: 0.8256 - val_loss: 0.6777 - val_accuracy: 0.8054
Epoch 4/10
1604/1604 [==============================] - 25s 15ms/step - loss: 0.5481 - accuracy: 0.8309 - val_loss: 0.6828 - val_accuracy: 0.7996
Epoch 5/10
1604/1604 [==============================] - 26s 16ms/step - loss: 0.5238 - accuracy: 0.8391 - val_loss: 0.6730 - val_accuracy: 0.8054
Epoch 6/10
1604/1604 [==============================] - 27s 17ms/step - loss: 0.4998 - accuracy: 0.8448 - val_loss: 0.6720 - val_accuracy: 0.8054
Epoch 7/10
1604/1604 [==============================] - 25s 16ms/step - loss: 0.4746 - accuracy: 0.8525 - val_loss: 0.7045 - val_accura

Save model

In [26]:
model.save('/Users/toni/Documents/my_model.h5')